In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'

import torch
import torch_geometric
from glob import glob
from tqdm.auto import tqdm, trange
import numpy as np
import matplotlib.pyplot as plt

import pytorch_lightning as pl
torch.set_printoptions(sci_mode=False)
# device = torch.device('cuda')
device = torch.device('cpu')
# pl.seed_everything(12345)

In [15]:
# Python version
# Loads data from files generated by the ATIS CAMERA
# This function only read (t,x,y,p) and discard other fields (if any) of events.
# timestamps are in uS
# td_data is a numpy array; x, y, p, t are td_data[:,0], td_data[:,1], td_data[:,2], td_data[:,3]
# 
# raw event format ([63:0]) (little-e): t(32bit)/p(4bit)/y(14bit)/x(14bit)
# t [63:32] / p [31:28] / y [27:14] / x [13:0]
#
# flipX, flipY allow to flip the image arround the X and Y axes. If these values
# are non zero, the corresponding dimension will be flipped considering its size
# to be the value contained in the 'flip' variable (i.e. X = flipX - X)
# (They defaults to 0 if non-specified)

def load_atis_data(filename: str, flipX: int = 0, flipY: int = 0):
    def parse_head(filename: str):
        from math import floor
        import struct
        
        f = open(filename, 'rb')

        # Parse header if any
        header = []
        endOfHeader = 0
        numCommentLine = 0
        while (endOfHeader == 0):
            bod = f.tell()
            tline = f.readline(256)

            if (tline[0] != ord('%')):
                endOfHeader = 1
            else:
                tline_str = tline.decode('ascii')
                words = tline_str.split()
                if (len(words) > 2 ):
                    if (words[1] == 'Date'):
                        if (len(words) > 3):
                            date = words[2] + ' ' + words[3]
                            header.append([words[1], date])
                    else:
                        header.append([words[1], words[2]])
                numCommentLine = numCommentLine + 1

        f.seek(bod, 0) # seek from the start of the file

        evType = 0
        evSize = 8
        if (numCommentLine > 0): # Ensure compatibility with previous files.
            # Read event type
            evType = struct.unpack('B', f.read(1))[0]
            # Read event size
            evSize = struct.unpack('B', f.read(1))[0]

        bof = f.tell() # from the start of event data
        f.seek(0,2) # move to the end of the file
        numEvents = floor((f.tell()-bof)/evSize)

        f.close()
        return numEvents, bof
    

    # read data
    import numpy as np
    numEvents, bof = parse_head(filename)
    raw_data = np.fromfile(filename, dtype=np.uint32, offset=bof).reshape((-1,2))
    allTs = raw_data[:, 0]
    allAddr = raw_data[:, 1]

    td_data = np.empty((numEvents, 4), dtype=np.uint32) # x y p t
    td_data[:, 3] = allTs

    xmask = int('00003FFF', 16)
    ymask = int('0FFFC000', 16)
    polmask = int('10000000', 16)
    xshift = 0 # bits to shift x to right
    yshift = 14 # bits to shift y to right
    polshift = 28 # bits to shift p to right

    addr = np.abs(allAddr) # make sure nonnegative or an error will result from bitand (glitches can somehow result in negative addressses...)
    td_data[:, 0] = (addr & xmask) >> xshift # x addresses
    td_data[:, 1] = (addr & ymask) >> yshift # y addresses
    td_data[:, 2] = (addr & polmask) >> polshift # 1 for ON, 0 for OFF

    if (flipX > 0):
        td_data[:, 0] = flipX - td_data[:, 0]
    if (flipY > 0):
        td_data[:, 1] = flipY - td_data[:, 1]

    return td_data


filename = '/users/yyang22/thesis/aegnn_project/obj_004396_td.dat'
td_data = load_atis_data(filename, flipX=0, flipY=0)
print(td_data)

[[   34    30     0     0]
 [   15    16     0    11]
 [   13    21     1    16]
 ...
 [   48    45     0 99873]
 [   29    16     0 99875]
 [   16    19     0 99916]]


In [32]:
import numpy as np

t = td_data[:,3]
t_shift = np.zeros_like(t)
t_shift[:-1] = t[1:]
t_shift[-1] = t[-1] * 10
dt = t_shift - t
idx_0 = np.nonzero(dt==0)[0]

max_same_t = 2
local_max_same_t = 2
idx_old = 0
for idx in idx_0:
    if idx == idx_old + 1:
        local_max_same_t += 1
        max_same_t = np.maximum(max_same_t, local_max_same_t)
    else:
        local_max_same_t = 2
    idx_old = idx
print(max_same_t)


2
